## SMAVA

Test exercise

In [1]:
# load in packages
library(caret)
library(ranger)
library(tidyverse)
library(e1071)
getwd()

Loading required package: lattice

Loading required package: ggplot2

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v tibble  2.1.3     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
v purrr   0.3.3     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x purrr::lift()   masks caret::lift()



[1] "/misc/build/0/pyeg0/caret/classr1"

In [2]:
load("bak/in/train.rdata")
load("bak/in/test.rdata")
ls()
# rm(train0)
train0 <- data.frame(train)
head(train)

# Carry some properties
smava0 <- list()

[1] "test"  "train"

customerNumber,bank,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,accepted,interestRate
<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
7544,B14,2.6820549,11.100511,261.0908,4.0264718,-1.4870793,FBW,M98CB,2.4194603,0.4615068,0.2702748,NO,NA
9909,B4,1.1749983,6.838129,276.1609,-3.2329746,-4.9360487,ACS,U5D5G,-7.6339564,0.8943440,0.2506165,NO,NA
24088,B25,1.8947994,NA,188.3505,2.1999443,-3.3122378,FBW,YVNYM,-1.3734885,0.6615393,0.3891396,YES,2.259308
12700,B3,1.7044089,5.412387,196.7486,-1.2095107,1.8524490,FBW,PBSEO,0.7316942,0.2678961,0.3731034,NO,NA
17866,B1,2.5554689,3.211833,168.7244,2.4950889,0.8707447,FBW,IQ4NP,2.7717305,0.2003426,0.4293550,NO,NA
18829,B8,0.2704093,NA,220.3993,0.9586655,3.4161934,ACS,GPX9G,4.6333058,0.3485185,0.3314056,NO,NA


In [3]:
# describe() isn't available
sapply(train, class)
sapply(train, summary)

customerNumber           bank             x1             x2             x3 
     "integer"       "factor"      "numeric"      "numeric"      "numeric" 
            x4             x5             x6             x7             x8 
     "numeric"      "numeric"       "factor"       "factor"      "numeric" 
            x9            x10       accepted   interestRate 
     "numeric"      "numeric"       "factor"      "numeric"

$customerNumber
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      2    7551   15105   15054   22518   29997 

$bank
  B1  B10  B11  B12  B13  B14  B15  B16  B17  B18  B19   B2  B20  B21  B22  B23 
4603 4495 4424 4509 4479 4492 4418 4547 4565 4402 4410 4541 4444 4496 4504 4522 
 B24  B25  B26  B27  B28  B29   B3  B30   B4   B5   B6   B7   B8   B9 
4518 4513 4569 4473 4565 4444 4480 4463 4424 4508 4479 4542 4588 4552 

$x1
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 0.02433  0.49992  1.00460  1.65895  1.95799 75.79947 

$x2
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   0.01    0.97    2.71    8.17    7.36  910.95   40520 

$x3
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  13.35  166.00  199.11  199.34  232.74  382.41 

$x4
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-13.5489  -0.7509   1.9963   1.9581   4.6776  18.8855 

$x5
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-13.6414  -0.7914   1.9706   1.9340   4.6317  18.2838 

$x6
  A

## Data analysis

The outcome variable accepted is balanced. So need to upsample.

customerNumber has 15000 unique.

NA's on interestRate, and x2. x2 is customer-based.

interestRate is for regression analysis later

In [4]:
fnas <- function(x) sum(as.integer(is.na(train[[x]])))
fnas("x2")
fnas("interestRate")

[1] 40520

[1] 68364

In [5]:
## interestRate is only given for accepted
train[train$accepted == "NO" && !is.na(train$interestRate), c("accepted", "interestRate")]

# No obvious rule
# sapply(train[is.na(train$x2),], summary)
# sapply(train[!is.na(train$x2),], summary)

accepted,interestRate
<fct>,<dbl>


In [6]:
## x2 is customer-based
train1 <- train[order(train$customerNumber, train$bank),]
train2 <- train1 %>%
select(customerNumber, x2) %>%
group_by(customerNumber) %>%
summarise(n = n(), na0 = sum(is.na(x2)))

In [7]:
# And is all NA at all banks for a particular type of customer.
train2[ (train2$n != train2$na0) && (train2$na0 > 0),]

customerNumber,n,na0
<int>,<int>,<int>


In [8]:
## capture those customers

smava0$"null-customer" <- train2[train2$na0 > 0, "customerNumber"][["customerNumber"]]
length(smava0$"null-customer")

[1] 4488

In [9]:
colnames(train1)
smava0$results <- train1[c("accepted", "interestRate")]

[1] "customerNumber" "bank"           "x1"             "x2"            
 [5] "x3"             "x4"             "x5"             "x6"            
 [9] "x7"             "x8"             "x9"             "x10"           
[13] "accepted"       "interestRate"

In [10]:
smava0

accepted,interestRate
<fct>,<dbl>
YES,3.315310
YES,5.746954
NO,NA
NO,NA
YES,4.248454
YES,3.542020
YES,2.030626
NO,NA
YES,2.630021


In [11]:
f0 <- c('bank', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10')

In [12]:
smava0$ft0 <- f0

In [13]:
save(train1, smava0, file="smava0.dat")

In [14]:
train2 <- train1 %>%
select(smava0$ft0)

In [16]:
train2[["x2"]] <- NULL

In [18]:
save(train2, smava0, file="smava02.dat")

In [ ]:
## Look at the test data
load("bak/in/test.rdata", envir=.GlobalEnv)
test

In [5]:
ctest <- unique(test$customerNumber)
ctrain <- unique(train$customerNumber)

In [10]:
length(ctrain)

[1] 14999

In [11]:
length(ctest)

[1] 15000